<a href="https://colab.research.google.com/github/SanjanaMohan34/mgmt467-analytics-portfolio/blob/Labs/Lab1_AI_Assisted_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 1: AI‑Assisted SQL Foundations (Colab)**
**Course:** MGMT 467 — AI‑Assisted Big Data Analytics in the Cloud  
**When:** Unit 1 · Week 3 (Thursday)  
**Goal:** Use *Gemini as a co‑pilot* to write and understand basic SQL against the **Superstore** dataset in **BigQuery**.  
**You will practice:** Prompt engineering → SQL generation → Query execution → Interpretation.

> ✅ **Deliverable (submit link in Brightspace):** A completed `Lab1_AI_Assisted_SQL.ipynb` with prompts, code, outputs.


## ✅ What you need before starting
- A Google account with access to **Google Colab**.
- A **Google Cloud Project** with **BigQuery** API enabled.
- A dataset named `superstore_data` and a table named `sales` (your instructor or TA will provide details).
- (Optional) A GitHub repo to store your work for your team.


## 0) Connect to class GitHub repo
## Install and import libraries
## Class Repo: bigDataNCloud/mgmt_467_resources

In [5]:
# # Replace with your actual class repo URL
# CLASS_REPO_URL = "https://github.com/SanjanaMohan34/mgmt467-analytics-portfolio"
# CLASS_REPO_DIR = "/content/class-repo"

# # If already cloned, skip re-downloading
# import os
# if not os.path.exists(CLASS_REPO_DIR):
#     !git clone {CLASS_REPO_URL} {CLASS_REPO_DIR}
# else:
#     %cd {CLASS_REPO_DIR}
#     !git pull

# # Move into repo folder
# %cd {CLASS_REPO_DIR}

# print("✅ Class repository ready at", CLASS_REPO_DIR)

In [6]:
!pip -q install google-cloud-bigquery google-cloud-bigquery-storage db-dtypes pandas pyarrow kagglehub

In [7]:
from google.cloud import bigquery
from google.colab import auth
import pandas as pd
import os, json, textwrap, pathlib, pprint
print("Libraries imported.")

Libraries imported.


## 1) Authenticate to GCP and set your project
Run the cell below. When prompted, authorize Colab to access your Google Cloud resources.

In [8]:
auth.authenticate_user()
print("✅ Authenticated.")

# Set your GCP project (EDIT THIS!)
PROJECT_ID = "mgmt467-project1"   # <-- EDIT ME
assert PROJECT_ID != "YOUR_PROJECT_ID_HERE", "Please set PROJECT_ID to your GCP project id."
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

# BigQuery client
bq = bigquery.Client(project=PROJECT_ID)
print("✅ BigQuery client created for:", PROJECT_ID)

✅ Authenticated.
✅ BigQuery client created for: mgmt467-project1


### Quick connection test (optional)
This will try to read the first 5 rows from your Superstore table. If it fails, double‑check your dataset/table name.

In [9]:
DATASET = "lab1_foundation"
TABLE   = "SuperStore"
BQ_TABLE = f"`{PROJECT_ID}.{DATASET}.{TABLE}`"

query = f"SELECT * FROM {BQ_TABLE} LIMIT 5"
try:
    df_preview = bq.query(query).to_dataframe()
    display(df_preview)
    print("✅ Connection OK.")
except Exception as e:
    print("⚠️ Could not query the table. Error below:")
    print(e)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,5769,CA-2015-154900,2015-02-25,2015-03-01,Standard Class,SS-20875,Sung Shariari,Consumer,United States,Leominster,...,1453,East,OFF-LA-10001641,Office Supplies,Labels,Avery 518,3.15,1,0.0,1.5120
1,5770,CA-2015-154900,2015-02-25,2015-03-01,Standard Class,SS-20875,Sung Shariari,Consumer,United States,Leominster,...,1453,East,OFF-PA-10002377,Office Supplies,Paper,Adams Telephone Message Book W/Dividers/Space ...,22.72,4,0.0,10.2240
2,9028,US-2016-152415,2016-09-17,2016-09-22,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,Marlborough,...,1752,East,FUR-FU-10002597,Furniture,Furnishings,"C-Line Magnetic Cubicle Keepers, Clear Polypro...",14.82,3,0.0,6.2244
3,9029,US-2016-152415,2016-09-17,2016-09-22,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,Marlborough,...,1752,East,FUR-FU-10004864,Furniture,Furnishings,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",191.82,3,0.0,61.3824
4,8332,CA-2016-153269,2016-03-09,2016-03-12,First Class,PS-18760,Pamela Stobb,Consumer,United States,Andover,...,1810,East,OFF-ST-10004634,Office Supplies,Storage,"Personal Folder Holder, Ebony",11.21,1,0.0,3.3630


✅ Connection OK.


## 2) (Optional) Download a dataset with **KaggleHub**
Use **KaggleHub** to pull sample data locally into Colab (e.g., for offline exploration or to stage for GCS).
> This is **optional** for Lab 1 (we focus on BigQuery), but useful to practice data access workflows.
> https://www.kaggle.com/datasets/anandaramg/global-superstore

In [10]:
import kagglehub, os, pathlib

DATASET_REF = "anandaramg/global-superstore"   # you can change this to any public dataset reference

download_path = kagglehub.dataset_download(DATASET_REF)
print("Path to dataset files:", download_path)

# List files
p = pathlib.Path(download_path)
print("Files:")
for f in p.glob("**/*"):
    if f.is_file():
        print("-", f)


100%|██████████| 3.18M/3.18M [00:00<00:00, 57.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/anandaramg/global-superstore/versions/1
Files:
- /root/.cache/kagglehub/datasets/anandaramg/global-superstore/versions/1/Global Superstore.txt


## 3) Prompting approach for Lab 1
You'll **paste prompts into Gemini** (in a separate tab), get back **SQL**, then paste that SQL here to execute.  
We’ll practice three core question types:
- **“What”** (lists & filters) → `SELECT`, `WHERE`, `DISTINCT`
- **“How many”** (counts by category) → `COUNT`, `GROUP BY`
- **“Who is best”** (rank/limit by metric) → `SUM`, `ORDER BY`, `LIMIT`

### 3A) “What” Question — `SELECT`, `WHERE`, `DISTINCT`
**Business Question:** *A manager wants a list of all **unique** product sub‑categories sold in the **West** region.*

**Paste this prompt into Gemini (no edits needed except the project id if you used a different table path):**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`.
# GOAL: Find all the unique values in the 'Sub_Category' column, but only for rows where the 'Region' column is exactly 'West'.
```

**Then paste the SQL you get into the cell below** (replace the placeholder) and run it.

In [11]:
sql_str = f"""
SELECT DISTINCT `Sub-Category`
FROM `{PROJECT_ID}.lab1_foundation.SuperStore`
WHERE Region = 'West'
ORDER BY `Sub-Category`
"""
# This line is no longer needed because we are using an f-string
# sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)

df_what = bq.query(sql_str).to_dataframe()
print(f"Rows: {len(df_what)}")
display(df_what.head(20))

Rows: 17


,Sub-Category
0,Accessories
1,Appliances
2,Art
3,Binders
4,Bookcases
5,Chairs
6,Copiers
7,Envelopes
8,Fasteners
9,Furnishings


**Explain it back (metacognition):** In Gemini, paste:  
> *Explain the following SQL query line by line: [paste your SQL]*  

SELECT DISTINCT `Sub-Category`
This line selects the column named Sub-Category. The DISTINCT keyword ensures that only unique values from this column are returned, removing any duplicates.

FROM `{PROJECT_ID}.lab1_foundation.SuperStore`
This line specifies the table from which to retrieve the data. It's referencing the SuperStore table within the lab1_foundation dataset in your specified Google Cloud project ({PROJECT_ID}).

WHERE Region = 'West'
This line filters the rows from the table. It only includes rows where the value in the Region column is exactly 'West'.

ORDER BY `Sub-Category`
This line sorts the resulting unique sub-categories alphabetically in ascending order.

In summary, this query retrieves all the unique product sub-categories from the 'West' region of your Superstore data and presents them in alphabetical order.

Summarize what you learned here:

> **Notes:** SELECT DISTINCT 'Sub-Category' helps up select all the unique values in the Sub-Category column. The next line using the FROM query helps us specify the location of the data which we want to access. The WHERE Region = 'West' query helps us choose the Sub-Categories only from the rows where the region in west. Finally, the ORDER BY query organises and displays the output (west region sub-categories) in alphabetical, ascending order. If we want to have the output in descending order, we would have to specify DESC in the last query line.

### 3B) “How many” Question — `COUNT`, `GROUP BY`
**Business Question:** *How many orders were placed in each **Ship Mode**?*

**Gemini prompt:**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`.
# GOAL: Count the total number of records for each unique value in the 'Ship_Mode' column.
# The final result should have two columns: 'Ship_Mode' and 'order_count'.
```


In [12]:
sql_str = f"""
SELECT `Ship Mode`, COUNT(*) AS order_count
FROM `{PROJECT_ID}.lab1_foundation.SuperStore`
GROUP BY `Ship Mode`
ORDER BY order_count DESC
"""
# The line below is no longer needed as the f-string handles the project ID
# sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)

df_howmany = bq.query(sql_str).to_dataframe()
display(df_howmany)

,Ship Mode,order_count
0,Standard Class,5968
1,Second Class,1945
2,First Class,1538
3,Same Day,543


**Gemini Output**: SELECT `Ship Mode`, COUNT(*) AS order_count
This line selects two things: the Ship Mode column and a count of all rows for each group (which we'll define in the GROUP BY clause). COUNT(*) counts all rows, and AS order_count gives this count a new name, order_count, for the output.

FROM `{PROJECT_ID}.lab1_foundation.SuperStore`
This line specifies the table the data is coming from, which is the SuperStore table in your lab1_foundation dataset within your GCP project.

GROUP BY `Ship Mode`
This is a crucial line. It groups the rows based on the unique values in the Ship Mode column. The COUNT(*) function in the SELECT statement then counts the rows within each of these groups.

ORDER BY order_count DESC
This line sorts the final results based on the order_count in descending order (DESC), so the ship mode with the highest number of orders appears first.

In summary, this query counts how many orders fall into each 'Ship Mode' category and then lists these counts from highest to lowest.

**My Notes:** SELECT 'Ship Mode`, COUNT(*) AS order_count selects data from the Ship Mode column and counts the number of rows with the same ship mode and saves this number in a new column, order_count. The FROM query defines where the data we're working with is location. The GROUP BY query defines which colum/values the data should be grouped by which is ship mode in this case. Finally we use ORDER BY to organise the output by order_count in descending order. Even though we define what to group the data by after we use the count function, SQL processes the lines in a sequence that makes logical sense to it so it does FROM, WHERE, GROUP BY, HAVING and then ORDER BY.

### 3C) “Who is best” Question — `SUM`, `ORDER BY`, `LIMIT`
**Business Question:** *Identify the **top 5 most profitable customers**.*

**Gemini prompt:**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`. The customer identifier is 'Customer_ID'.
# GOAL: Calculate the sum of 'Profit' for each customer. The final output should show the 'Customer_ID' and their total profit, sorted from highest to lowest profit, and limited to only the top 5 results.
```


In [13]:
sql_str = f"""
SELECT `Customer ID`, SUM(Profit) AS total_profit
FROM `{PROJECT_ID}.lab1_foundation.SuperStore`
GROUP BY `Customer ID`
ORDER BY total_profit DESC
LIMIT 5
"""
# sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)

df_best = bq.query(sql_str).to_dataframe()
display(df_best)

,Customer ID,total_profit
0,TC-20980,8981.3239
1,RB-19360,6976.0959
2,SC-20095,5757.4119
3,HL-15040,5622.4292
4,AB-10105,5444.8055


**Gemini Output:** SELECT `Customer ID`, SUM(Profit) AS total_profit
This line selects two things: the Customer ID column and the sum of the Profit for each group (which we'll define in the GROUP BY clause). SUM(Profit) calculates the total profit, and AS total_profit gives this sum a new name, total_profit, for the output.

FROM `{PROJECT_ID}.lab1_foundation.SuperStore`
This line specifies the table the data is coming from, which is the SuperStore table in your lab1_foundation dataset within your GCP project.

GROUP BY `Customer ID`
This is a crucial line. It groups the rows based on the unique values in the Customer ID column. The SUM(Profit) function in the SELECT statement then calculates the total profit for each of these customer groups.

ORDER BY total_profit DESC
This line sorts the final results based on the total_profit in descending order (DESC), so the customer with the highest total profit appears first.

LIMIT 5
This line restricts the output to only the top 5 rows after the sorting has been applied.

In summary, this query calculates the total profit for each customer, sorts them from highest to lowest total profit, and then shows only the top 5 customers.

**My Notes:** The first line of code selects the customer id column and uses the sum function to add all the values in the profit columns for the same customer and saves this value into a new column 'total_profit'. Then we use the from function to define where the data is stored/can be accessed from. Then we use the group by function to group the values by customer id and ask it to order the output by the total_profit in descending order. We use LIMIT to get the first 5 values of the output to define the top 5 customers by total profit made from each customer.

## 4) Challenge prompts (author your own)
Write **your own precise prompt** in a text cell (or comment) for each question below, then get SQL from Gemini and run it in the provided code cells.

**Challenge 1:** *What is the **average discount** for products in the **Technology** category sold in the **East** region?*  
**Challenge 2:** *How many **unique customers** has each **Segment** (Consumer/Corporate/Home Office) served?*

In [14]:
sql_str = f"""
-- PASTE your Gemini-generated SQL here
SELECT Category, Region, AVG(Discount) AS avg_discount
FROM `{PROJECT_ID}.lab1_foundation.SuperStore`
WHERE Category = 'Technology' AND Region = 'East'
GROUP BY Category, Region
"""
# sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)
df_ch1 = bq.query(sql_str).to_dataframe()
display(df_ch1)

,Category,Region,avg_discount
0,Technology,East,0.143364


**My Prompt:** Task: Generate a BigQuery SQL query.

Context: The table is `{PROJECT_ID}.lab1_foundation.SuperStore` The customer identifier is Category.

Goal: to calculate the average ‘Discount’ for products in the ‘Technology’ category sold in the East ‘Region’. The final output should show the ‘Category’, ‘Region’, and the calculated average discount.

**Notes:** The code selects the category and region columns and specifically choses the rows that correspond to technology and east respectively. It then takes the average of all of the discounts of these rows and puts that in a new column avg_discount and uses group by to display the output.


In [15]:
sql_str = f"""
-- PASTE your Gemini-generated SQL here
SELECT Segment, COUNT(DISTINCT `Customer ID`) AS unique_customers
FROM `{PROJECT_ID}.lab1_foundation.SuperStore`
GROUP BY Segment
ORDER BY unique_customers DESC
"""
# sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID) # This line is no longer needed

df_ch2 = bq.query(sql_str).to_dataframe()
display(df_ch2)

,Segment,unique_customers
0,Consumer,409
1,Corporate,236
2,Home Office,148


**My Prompt:** Task: Generate a BigQuery SQL query.

Context: The table is {PROJECT_ID}.lab1_foundation.SuperStore The customer identifier is Segment.

Goal: to calculate the number of unique customers in each segment. The final output should show the segment and the number of unique customers in descending order.

**Notes:** The code firsts tells the location of the data then it groups it by the segment. After the data is grouped by segment, it counts the number of unique customers by using the customer id column and puts that in a new column, unique_customers. Finally it shows the output after organising in in descending order by the unique_customers column.

## 5) Reflection (DIVE mindset)
- **Discover:** What did you find first?
When I first gave gemini the promt and ran the code, I found that it was really good at gaining context and fitting the code according to the cells before and after. It would also use this context and output from previous cells to fix any errors in the code. I also found that it's best to be really specific in what the output should be to guide it better to give the answer we're looking for.

- **Investigate:** What alternate query or filter changed the story?  
When I tried to write my first prompt, I didn't specify what I wanted the output to be. But once i started doing that, it gave me the exact output I was looking for on the first try. I also found that following the professor's way of defining the task, context and goal separately streamlines the prompt engineering process better.

- **Validate:** Where could the AI‑generated SQL be wrong or incomplete? How did you check?
Most of the times when the AI generated SQL ran into errors was with the column names and names of values. I used the preview dataset on the top of this notebook to cross check the names of the columns. Other than that, I was a little confused with how SQL code was organised in terms of the lines that made it group the column values and such because those came after we already told it to sum the values. When I realised this, I asked gemini to explain so I would understand what it was doing better. On top of questioning things that struck me as odd, I would also ask it to explain every line of code so I knew exactly what was going on in each cell.

- **Extend:** Which stakeholder could use your results tomorrow? What action should they take?
A business owner/manager could use my results. They would be able to tell the top 5 customers to sell to, how fast customers want the items to be shipped to them, the specific products sold in the west region, average discounts for technology in the east region as well as the unique customers in each segment.

Based on this data, the business owner/manager would be able to make more targetted marketing plans. They would know who their top 5 customers are to cater to them more as well as also focus on targeting the customers in the mid range to make them buy more. On top of this, the manager/owner would be able to keep the products bought the most in stock so they don't run out as well as make sure to keep account of which items sell the most in which region. Additionally the business owner would be able to target the various customer segments such that they cater to their most profit inducing customers the most and also focus on urging their second top selling category (corporate) to buy more through targetted marketing.

## 6) Save your work to GitHub (pick one of the options)
**Option A (recommended):** In Colab, go to **File → Save a copy in GitHub…** and select your team repo + folder (e.g., `labs/Unit1/`).  
**Option B (CLI, if you know git):**
```bash
# (In Colab) mount Drive, then clone/pull/push as usual with a PAT
# Be careful to NOT store secrets in the notebook.
```
Name the file **`Lab1_AI_Assisted_SQL.ipynb`** and push it to your team repo.